In [1]:
# Feature Engineering und Datensatzaufbau

import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))

In [2]:
from src.data import download_data
print("Module erfolgreich importiert")

Module erfolgreich importiert
